In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(23)
tf.random.set_seed(23)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


## Tensorflow Common Types and Operations

#### Constants and Operations

In [5]:
tf_array = tf.constant([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])
tf_array

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>

In [6]:
tf_const = tf.constant(31)
tf_const

<tf.Tensor: shape=(), dtype=int32, numpy=31>

In [7]:
tf_array[:,0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 4., 7.], dtype=float32)>

In [8]:
tf_array.shape

TensorShape([3, 3])

In [9]:
tf_array + 1

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 2.,  3.,  4.],
       [ 5.,  6.,  7.],
       [ 8.,  9., 10.]], dtype=float32)>

In [10]:
tf_array @ tf_array

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 30.,  36.,  42.],
       [ 66.,  81.,  96.],
       [102., 126., 150.]], dtype=float32)>

#### Tensforflow and Numpy

In [12]:
tf_const

<tf.Tensor: shape=(), dtype=int32, numpy=31>

In [13]:
tf_const.numpy()

31

In [14]:
tf_array.numpy()

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)

In [15]:
type(tf_array.numpy())

numpy.ndarray

In [16]:
numpy_array = np.array([[1, 1, 2], [3, 5 ,8]])
numpy_array

array([[1, 1, 2],
       [3, 5, 8]])

In [17]:
tf.constant(numpy_array)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[1, 1, 2],
       [3, 5, 8]])>

#### Tensorflow Variables

In [18]:
tf_var = tf.Variable([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])
tf_var

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>

In [20]:
tf_var[:, 1].assign(tf_var[:, 1] + tf_var[:, 2])
tf_var

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.,  5.,  3.],
       [ 4., 11.,  6.],
       [ 7., 17.,  9.]], dtype=float32)>

#### Tensorflow Strings

In [21]:
tf_str = tf.constant("cheeto")
tf_str

<tf.Tensor: shape=(), dtype=string, numpy=b'cheeto'>

In [35]:
tf_chars = tf.constant([c for c in "cheeto"])
tf_chars

<tf.Tensor: shape=(6,), dtype=string, numpy=array([b'c', b'h', b'e', b'e', b't', b'o'], dtype=object)>

In [38]:
tf.strings.unicode_decode(tf_chars, "UTF-8")

<tf.RaggedTensor [[99],
 [104],
 [101],
 [101],
 [116],
 [111]]>

#### Tensorflow Sets

Each row in a matrix represents a set, so each set is a vector.

In [41]:
tf_set1 = tf.constant([[1, 3, 5, 7], [9, 11, 13, 15]])
tf_set2 = tf.constant([[0, 2, 5], [13, 8, 11]])
tf.sparse.to_dense(tf.sets.union(tf_set1, tf_set2))

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  5,  7],
       [ 8,  9, 11, 13, 15,  0]], dtype=int32)>

## Custom Activation Functions, Initializers, Metrics and Losses

In [42]:
def my_sigmoid(z):
    return 1. / (1. + tf.exp(-z))

In [45]:
my_sigmoid(0.1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5249792>

In [46]:
def my_glorot_initializer(shape):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=tf.float32)

In [50]:
glorot_weights = my_glorot_initializer((30, 8))
glorot_weights

<tf.Tensor: shape=(30, 8), dtype=float32, numpy=
array([[-2.08642241e-02, -1.77352726e-02,  1.98043808e-01,
        -2.75053501e-01, -1.01896167e-01,  1.23544201e-01,
         1.59893736e-01,  5.22514470e-02],
       [-1.67840973e-01,  6.09271899e-02, -2.68196225e-01,
         1.24674477e-01,  2.87606031e-01, -2.87363529e-01,
        -1.70807645e-01,  5.34604132e-01],
       [-3.20544004e-01, -1.63540497e-01,  1.78350285e-01,
         1.88906863e-02,  7.53221512e-02,  1.21620618e-01,
        -1.91055834e-02,  2.25163680e-02],
       [-6.04539923e-02,  3.76067683e-02, -3.02041352e-01,
        -8.94935727e-02, -3.00004184e-01,  2.42172152e-01,
         2.38217428e-01, -3.15907747e-01],
       [ 3.22891861e-01, -1.48134485e-01,  1.71841890e-01,
         2.91384645e-02, -2.04411522e-01, -2.77732126e-02,
        -1.54018924e-01,  8.77042636e-02],
       [-7.42704794e-02,  1.56097993e-01,  1.02842212e-01,
         3.54622960e-01, -2.56843008e-02, -1.69651330e-01,
         1.42681032e-01,  2.

In [53]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [54]:
my_l1_regularizer(glorot_weights)

<tf.Tensor: shape=(), dtype=float32, numpy=0.41253716>

In [61]:
def my_l2_regularizer(weights):
    return tf.reduce_sum(0.01 * tf.square(weights))

In [62]:
my_l2_regularizer(glorot_weights)

<tf.Tensor: shape=(), dtype=float32, numpy=0.10716777>

In [63]:
def my_relu(z):
    return tf.where(z < 0., tf.zeros_like(z), z)

In [64]:
my_relu(-3)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [65]:
my_relu(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [66]:
layer = keras.layers.Dense(30, activation=my_relu, 
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer)

In case we have hyperparameters, we need to define a class

In [67]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

* For a custom metric we need to inherit from `keras.metrics.Metric`.

* For a custom loss function we need to inherit from `keras.losses.Loss`.

## Custom Layers

In [ ]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}